In [19]:
import threading 
class A(threading.Thread):
    x = 0
    def __init__(self):
        threading.Thread.__init__(self)
    def add(self,n):
        global x
        x = x + n
#         x = x - n
        print(x,threading.currentThread())
class B(threading.Thread):
    x = 0
    def __init__(self):
        threading.Thread.__init__(self)
    def add(self,n):
        global x
        x = x + n**2
#         x = x - n**2
        print(x,threading.currentThread())
AA = [A() for i in range(4)]
BB = [B() for j in range(5)]
for a in AA:
    a.start()
    a.add(2)
for b in BB:
    b.start()
    b.add(10)

import time
time.sleep(0.1)

# for a in AA:
#     a.stop()
# for i in range(10):
#     for a in AA:
#         a.join()
#     # for b in BB:
#     #     b.stop()
#     for b in BB:
#         b.join()

2 <_MainThread(MainThread, started 10772)>
4 <_MainThread(MainThread, started 10772)>
6 <_MainThread(MainThread, started 10772)>
8 <_MainThread(MainThread, started 10772)>
108 <_MainThread(MainThread, started 10772)>
208 <_MainThread(MainThread, started 10772)>
308 <_MainThread(MainThread, started 10772)>
408 <_MainThread(MainThread, started 10772)>
508 <_MainThread(MainThread, started 10772)>


In [27]:
import threading
import numpy as np
a = np.array([[1,2,3],[3,4,5]])
a = np.vstack((a,[3,3,3]))

In [29]:
class StateProcessor():
    """
    Processes a raw Atari iamges. Resizes it and converts it to grayscale.
    图片的预处理。
    """
    def __init__(self):
        # Build the Tensorflow graph
        with tf.variable_scope("state_processor"):
            self.input_state = tf.placeholder(shape=[210, 160, 3], dtype=tf.uint8)
            self.output = tf.image.rgb_to_grayscale(self.input_state)
            self.output = tf.image.crop_to_bounding_box(self.output, 34, 0, 160, 160)
            self.output = tf.image.resize_images(
                self.output, [84, 84], method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
            self.output = tf.squeeze(self.output)

    def process(self, sess, state):
        """
        Args:
            sess: A Tensorflow session object
            state: A [210, 160, 3] Atari RGB State

        Returns:
            A processed [84, 84, 1] state representing grayscale values.
        """
        return sess.run(self.output, { self.input_state: state })

## Estimator
网络定义和Value、Policy的估计方法

In [4]:
import numpy as np
import tensorflow as tf 
def build_shared_network(X):
    conv1 = tf.layers.conv2d(X,filters = 16,kernal_size = 8,strides =4,
                             activation = tf.nn.relue,name = 'conv1')
    conv2 = tf.layers.conv2d(conv1,filters = 32, kernal_size = 4, strides = 2,
                            activation = tf.nn.relu, name = 'conv2')
    fc = tf.layers.dense(tf.layers.flatten(conv2),units = 256,activation = tf.nn.relu,name = 'fc')
    return fcs


### Policy Estimator

In [17]:
class Policy_Estimator():
    def __init__(self,num_outputs, reuse = False, trainable = True):
        """
            num_outputs : 输出单元数量
            resuse : 是否重使用当前存在的共享网络
            trainable : 不需要更新lochal model的Actor将其设置为False
        """
        self.num_outputs = num_outputs
        self.states = tf.placeholder(tf.uint8 ,shape = [None,84,84,4], name = 'X')
        self.targets = tf.placeholder(tf.float32,shape = [None],name = 'y') 
        self.actions = tf.placeholder(tf.int32,shape = [None], name = 'actions')
        
#         Normalize
        X = tf.to_float(self.states) / 255.0
        batch_szie  = tf.shape(self.states)[0]
#         build graph
        with tf.variable_scope('shared',reuse = reuse):
            fc1 = build_shared_network(X)
        with tf.variable_scope('policy_network'):
            self.logit = tf.layers.dense(inputs = fc1, units = num_outputs,activation = None)
            self.prob = tf.nn.softmax(self.logit) + 1e-8
        self.predict = {
            'logit':self.logit,
            'prob': self.prob
        }
#         add entropy
        self.entropy = tf.reduce_sum(self.logit * tf.log(self.logit),axis = 1,name = 'entropy')
        self.entropy_mean = tf.reduce_mean(self.entropy,name = 'entropy_mean')
        
        # Get the predictions for the chosen actions only
        gather_indices = tf.range(batch_size) * tf.shape(self.probs)[1] + self.actions
        self.picked_action_probs = tf.gather(tf.reshape(self.probs, [-1]), gather_indices)
        
        self.losses = -(tf.log(self.picked_action_probs)*self.targets + 0.01*self.entropy)
        self.loss = tf.reduce_sum(self.losses)
        
        if trainable:
            self.optimizer = tf.train.RMSPropOptimizer(0.00025,0.99, 0.0, 1e-6)
#             self.train_op = self.optimizer.minimize(self.loss)
            self.grads_and_vals = self.optimizer.compute_gradients(self.loss)
            self.grads_and_vals = [[grad,val] for grad ,val in self.grads_and_vals if grad is not None]
            self.train_op = self.optimizer.apply_gradients(self.grads_and_vars,
                                                           global_step=tf.train.get_global_step())

### Value Estimator

In [18]:
class Value_Estimator():
    def __init__(self,resuse = False,trainable = True):
        self.states = tf.placeholder(tf.uint8 ,shape = [None,84,84,4], name = 'X')
        self.targets = tf.placeholder(tf.float32,shape = [None],name = 'y') 
        
        X = tf.to_float(self.states) / 255.0
        batch_szie  = tf.shape(self.states)[0]
        
        with tf.variable_scope('shared',reuse = reuse):
            fc1 = build_shared_network(X)
        with tf.variable_scope('Value_network'):
            self.logit = tf.layers.dense(fc1,1,activation = None)
            self.logit = tf.squeeze(self.logit,squeeze_dim = [1],name = 'logit')
            self.losses = tf.squared_difference(self.logit,self.targets)
            self.loss = tf.reduce_sum(self.losses,name = 'loss')
            self.predictions = {
                'logit':self.logit
            }
        if trainable :
            self.optimizer = tf.train.RMSPropOptimizer(0.00025,0.99, 0.0, 1e-6)
            self.grads_and_vals = self.optimizer.compute_gradients(self.loss)
            self.grads_and_vals = [[grad,val] for grad ,val in self.grads_and_vals if grad is not None]
            self.train_op = self.optimizer.apply_gradients(self.grads_and_vars,
                                                           global_step=tf.train.get_global_step())

## Worker

In [31]:
import collections
import itertools
Transition = collections.namedtuple("Transition", ["state", "action", "reward", "next_state", "done"])
def make_copy_variables_op(v1_list,v2_list):
    """
        这是一个交换拷贝Variable类型的函数,返回这些操作（op）的集合
    """
    v1_list = list(sorted(v1_list,key = lambda v:v.name))
    v2_list = list(sorted(v2_list,key = lambda v:v.name))
    update_op = []
    for v1,v2 in zip(v1_list,v2_list):
        op = v2.assign(v1)
        update_op.append(op)
    return update_op
def make_train_op(local_estimator, global_estimator):
    """
        create an operation that applies local estimator's gradient to global estimator
    """
    local_grad ,_ = zip(*local_estimator.grads_and_vals)
    local_grad = tf.clip_by_global_norm(local_grad,5.0) # 调整输入的2范式值不能超过某值
    _, global_var = zip(*global_estimator.grads_and_vals)
    local_grad_and_global_var = list(zip(local_grad,global_var))
    return global_estimator.optimizer.apply_gradients(local_global_grads_and_vars,
          global_step=tf.train.get_global_step())

In [30]:
class Worker():
    """
        An A3C worker run episode locally and update global shared policy and value network
    """
    def __init__(self,name,env,policy_net,value_net,global_counter,reward_discount=0.99,max_global_steps =None):
        """
            name : 每个worker的命名
            policy_net : instance of global shared policy net
            global_count :  Iterator that holds the global step
            max_global_steps : If set, stop coordinator when global_counter > max_global_steps
        """
        self.name = name
        self.env = env
        self.global_policy_net = policy_net
        self.global_value_net = value_net
        self.global_counter = global_counter
        self.global_step = tf.train.get_global_step()
        self.local_counter = itertools.count()
        self.sp = StateProcessor()
        
#         创建不异步更新的net(local)
        with tf.variable_scope(name):
            self.policy_net = Policy_Estimator(policy_net.num_outputs)
            self.value_net = Value_Estimator(reuse = True)
        # Op to copy params from global policy/valuenets
#         将全局参数给local
        self.copy_params_op = make_copy_params_op(
          tf.contrib.slim.get_variables(scope="global", collection=tf.GraphKeys.TRAINABLE_VARIABLES),
          tf.contrib.slim.get_variables(scope=self.name+'/', collection=tf.GraphKeys.TRAINABLE_VARIABLES))
        
        self.pnet_train_op = make_train_op(self.policy_net,self.global_policy_net)
        self.vnet_train_op = make_train_op(self.value_net,self.global_value_net)
        
        self.state = None
    def run():
        with sess.as_default(), sess.graph.as_default():
      # Initial state
          self.state = atari_helpers.atari_make_initial_state(self.sp.process(self.env.reset()))
        try:
            while not coord.should_stop():
          # Copy Parameters from the global networks
            sess.run(self.copy_params_op)

          # Collect some experience
            transitions, local_t, global_t = self.run_n_steps(t_max, sess)
            
            if self.max_global_steps is not None and global_t >= self.max_global_steps:
                tf.logging.info("Reached global step {}. Stopping.".format(global_t))
                coord.request_stop()
            return

          # Update the global networks
            self.update(transitions, sess)
        except tf.errors.CancelledError:
        return
    
    def _policy_net_predict(self, state, sess):
        feed_dict = { self.policy_net.states: [state] }
        preds = sess.run(self.policy_net.predictions, feed_dict)
        return preds["probs"][0]

    def _value_net_predict(self, state, sess):
        feed_dict = { self.value_net.states: [state] }
        preds = sess.run(self.value_net.predictions, feed_dict)
        return preds["logits"][0]
    def run_n_steps(self,n,sess):
        transitions = []
        for _ in range(n):
            action_probs = self._policy_net_predict(self.state,sess)
            action = np.random.choice(np.arange(len(action_probs)),p = action_probs)
            next_state, reward,done, _ = self.env.step(action)
            next_state = self.sp.process(next_state)
            trainsitions.append(Transition(state = self.state,action = self.action,
                                           reward = reward,next_state = next_state,done = done))
            local_t = next(self.local_counter)
            global_t = next(self.global_counter)
        if local_t % 100 == 0:
            tf.logging.info("{}: local Step {}, global step {}".format(self.name, local_t, global_t))

        if done:
            self.state = atari_helpers.atari_make_initial_state(self.sp.process(self.env.reset()))
            break
        else:
            self.state = next_state
    def update(transition,sess):
        """
            update global policy / value network
        """
            reward = 0.0
        if not transitions[-1].done:
            reward = self._value_net_predict(transitions[-1].next_state, sess)

    # Accumulate minibatch exmaples
        states = []
        policy_targets = []
        value_targets = []
        actions = []
        
        for transition in transitions[::-1]:
            reward = transition.reward + self.discount_factor * reward
            policy_target = (reward - self._value_net_predict(transition.state, sess))
      # Accumulate updates
            states.append(transition.state)
            actions.append(transition.action)
            policy_targets.append(policy_target)
            value_targets.append(reward)
        feed_dict = {
            self.policy_net.states: np.array(states),
            self.policy_net.targets: policy_targets,
            self.policy_net.actions: actions,
            self.value_net.states: np.array(states),
            self.value_net.targets: value_targets,
        }
        global_step, pnet_loss, vnet_loss, _, _ = sess.run([
            self.global_step,
            self.policy_net.loss,
            self.value_net.loss,
            self.pnet_train_op,
            self.vnet_train_op,
        ], feed_dict)
        return pnet_loss, vnet_loss

## Train(Brain)

In [35]:
import multiprocessing
import tensorflow as tf
NUM_WORKERS = multiprocessing.cpu_count()

In [34]:
with tf.device('/cpu:0'):
    global_step = tf.Variable(0, name="global_step", trainable=False)
#     global policy and value nets:
    with tf.variable_scope('global') as vs:
        policy_net = Policy_Estimator(num_outputs = NUM_OUTPUTS)
        value_net = Value_Estimator(reuse = True)
    global_counter = itertools.count()
    
    workers = []
    for worker_id in range(NUM_WORKERS):
        worker = Worker(
            name = 'Worker_{}'.format(worker_id),
            env = make_env(),
            policy_net = policy_net,
            value_net = value_net,
            global_counter = global_counter,
            discount_factor = 0.99,
        )
        workers.append(worker)
    saver = tf.train.Saver(keep_checkpoint_every_n_hours=2.0, max_to_keep=10)
    with tf.Session() as tf:
        sess.run(tf.global_variables_initializer())
        coord = tf.train.Coordinator()
        # Load a previous checkpoint if it exists
        latest_checkpoint = tf.train.latest_checkpoint(CHECKPOINT_DIR)
        worker_threads = []
        for worker in wokers:
            worker_fn = lambda worker=worker :worker.run(sess,coord,FlAGS.t_max)
            t = threading.Thread(target = worker_fn)
            t.start()
            worker_threads.append(t)

4